In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from utilities import format_batch_dcgan, manage_batch_2c
import statistics
import pretty_midi as pm
import pygame

2023-04-25 03:16:45.146596: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


pygame 2.3.0 (SDL 2.24.2, Python 3.10.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


/Users/benmodlin/miniconda3/envs/py4sci/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
generator = tf.keras.Sequential()
generator.add(layers.Dense(1250 * 11 * 256, use_bias=False, input_shape=(100,)))
generator.add(layers.BatchNormalization())
generator.add(layers.LeakyReLU())

generator.add(layers.Reshape((1250, 11, 256)))
# (2500, 11, 256)

generator.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
# (2500, 11, 128)

generator.add(layers.BatchNormalization())
generator.add(layers.LeakyReLU())

generator.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 1), padding='same', use_bias=False))
# (5000, 11, 64)

generator.add(layers.BatchNormalization())
generator.add(layers.LeakyReLU())

generator.add(layers.Conv2DTranspose(2, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
# (10000, 22, 2)


In [8]:
discriminator = tf.keras.Sequential()
discriminator.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                    input_shape=[5000, 22, 2]))
discriminator.add(layers.LeakyReLU())
discriminator.add(layers.Dropout(0.3))

discriminator.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
discriminator.add(layers.LeakyReLU())
discriminator.add(layers.Dropout(0.3))

discriminator.add(layers.Flatten())
discriminator.add(layers.Dense(1))

In [81]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    #print(real_output,fake_output)
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [82]:
noise_dim = 100
batch_size = 50

In [83]:
def train_dcgan(epochs):
    batch_manager = manage_batch_2c('trainfill5.pickle', batch_size)
    for i in range(epochs):
        batches = batch_manager.new_epoch()
        for batch in batches:
            formatted_batch = batch
            noise = tf.random.normal([batch_size, noise_dim])
            
            with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
                generated_images = generator(noise, training=True)

                real_output = discriminator(formatted_batch, training=True)
                fake_output = discriminator(generated_images, training=True)

                gen_loss = generator_loss(fake_output)
                disc_loss = discriminator_loss(real_output, fake_output)
                print(gen_loss, disc_loss)

            gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
            gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

            generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
            discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
        
        generator.save("./modelsave/gen" + str(28 + i+1) + ".h5")
        discriminator.save("./modelsave/dis" + str(28 + i+1) + ".h5")
        print("finished epoch!", i)


In [84]:
train_dcgan(25)
#generator.save("./modelsave/gen" + "25final" + ".h5")
#discriminator.save("./modelsave/dis" + "25final" + ".h5")

tf.Tensor(2.5077245, shape=(), dtype=float32) tf.Tensor(0.5798072, shape=(), dtype=float32)
tf.Tensor(3.982379e-06, shape=(), dtype=float32) tf.Tensor(12.895141, shape=(), dtype=float32)
tf.Tensor(0.4113873, shape=(), dtype=float32) tf.Tensor(1.3738706, shape=(), dtype=float32)
tf.Tensor(12.919416, shape=(), dtype=float32) tf.Tensor(3.9590256, shape=(), dtype=float32)
tf.Tensor(15.401765, shape=(), dtype=float32) tf.Tensor(6.796436, shape=(), dtype=float32)
tf.Tensor(11.441585, shape=(), dtype=float32) tf.Tensor(4.023897, shape=(), dtype=float32)
tf.Tensor(5.3105164, shape=(), dtype=float32) tf.Tensor(1.6013298, shape=(), dtype=float32)
tf.Tensor(0.29198056, shape=(), dtype=float32) tf.Tensor(1.6445138, shape=(), dtype=float32)
tf.Tensor(0.044939484, shape=(), dtype=float32) tf.Tensor(3.6573448, shape=(), dtype=float32)
tf.Tensor(0.15985912, shape=(), dtype=float32) tf.Tensor(2.2879338, shape=(), dtype=float32)
finished epoch! 0
tf.Tensor(1.4612933, shape=(), dtype=float32) tf.Tensor(0

KeyboardInterrupt: 

In [100]:
noise = tf.random.normal([1, noise_dim])
gen_r = generator(noise, training=False)
print(discriminator(gen_r, training=False))

tf.Tensor([[-14.8159075]], shape=(1, 1), dtype=float32)


In [91]:
print(gen_r.shape)

(1, 5000, 22, 2)


In [92]:
tones = [36, 38, 40, 37, 48, 50, 45, 47, 43, 58, 46, 26, 42, 22, 44, 49, 55, 57, 52, 51, 59, 53]
tone_map = {}
tones.sort()
time_resolution = 0.00089

for i in range(len(tones)):
    tone_map[i] = tones[i]

In [101]:
values = []
array = np.array(gen_r)
count = 0

test_array = np.zeros((5000, 22))
for i in range(5000):
    for j in range(22):
        value = array[0][i][j]
        values.append(value[0])
        
        if value[0] > 0:
            print(value[0])
            test_array[i][j] = (value[1] * 64) + 64
            print(j)
        

0.0078292815
4
0.015448219
4
0.008532749
4
0.0078736255
4
0.014251681
4
0.0032438072
4
0.015321752
4
0.033975
4
0.035480957
4
0.018557828
4
0.018739086
8
0.06137315
8
0.042631365
8
0.08586148
8
0.033520203
8
0.078552544
8
0.095992364
8
0.067473486
8
0.02378441
8


In [102]:
# create an empty PrettyMIDI object
midi = pm.PrettyMIDI()

# create an instrument object for drums
drums = pm.Instrument(program=0, is_drum=True)

# specify the tempo and time signature for the drum sequence
#midi.estimate_tempo()
#midi.time_signature_changes.append(pm.TimeSignature(4, 4))
tone_runs = [0] * 22
for i, row in enumerate(test_array):
    for tone, velocity in enumerate(row):
        if velocity != 0:
            if tone_runs[tone] != 0:
                tone_runs[tone][1] = i
            else:
                tone_runs[tone] = [i, i+1, velocity]
        else:
            tone_run_data = tone_runs[tone]
            if tone_run_data != 0:
                drums.notes.append(pm.Note(
                    velocity = int(tone_run_data[2]),
                    pitch = int(tone_map[tone]),
                    start = tone_run_data[0] * time_resolution,
                    end = tone_run_data[1] * time_resolution
                ))
            
            tone_runs[tone] = 0
            
for i, run in enumerate(tone_runs):
    if run != 0:
        tone_run_data = run
        drums.notes.append(pm.Note(
                        velocity = int(tone_run_data[2]),
                        pitch = int(tone_map[i]),
                        start = tone_run_data[0] * time_resolution,
                        end = tone_run_data[1] * time_resolution
                    ))

# add the drums instrument to the MIDI file
midi.instruments.append(drums)

# write the MIDI file to disk
midi.write('drums2.mid')


In [103]:
print([note for note in drums.notes])

[Note(start=0.045390, end=0.046280, pitch=38, velocity=45), Note(start=1.326990, end=1.327880, pitch=38, velocity=46), Note(start=1.330550, end=1.331440, pitch=38, velocity=46), Note(start=1.334110, end=1.335000, pitch=38, velocity=47), Note(start=1.337670, end=1.338560, pitch=38, velocity=47), Note(start=1.341230, end=1.342120, pitch=38, velocity=47), Note(start=1.344790, end=1.345680, pitch=38, velocity=47), Note(start=1.348350, end=1.349240, pitch=38, velocity=50), Note(start=1.351910, end=1.352800, pitch=38, velocity=49), Note(start=1.355470, end=1.356360, pitch=38, velocity=45), Note(start=1.504990, end=1.505880, pitch=44, velocity=50), Note(start=1.508550, end=1.509440, pitch=44, velocity=52), Note(start=1.512110, end=1.513000, pitch=44, velocity=52), Note(start=1.516560, end=1.517450, pitch=44, velocity=49), Note(start=1.520120, end=1.521010, pitch=44, velocity=47), Note(start=1.523680, end=1.524570, pitch=44, velocity=47)]
